<a href="https://colab.research.google.com/github/Dedaireks/ActNowRecSys/blob/master/Recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install numpy pandas surprise joblib


In [10]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

# Определение структуры тегов и подтегов
possible_tags = {
    "спорт": ["волейбол", "футбол", "теннис", "бильярд", "боулинг", "конный спорт", "хоббихорсинг", "фигурное катание", "спортзал", "автоспорт", "боевые искусства", "горнолыжный спорт", "бег", "велоспорт", "йога", "плавание"],
    "еда": ["ресторан (кафе)", "кофейня", "бар", "кулинария"],
    "развлечения": ["игры", "компьютерные игры", "кальян", "антикафе", "на свежем воздухе", "астрология", "книги", "животные", "рисование", "танцы", "фотография"],
    "культура": ["театр", "кинотеатр", "музей", "выставка", "музыка", "религия"],
    "образование": ["лекция", "митап", "конференция", "университет", "школа", "колледж", "тренинг", "курс"],
    "туризм": ["горы", "море", "пустыня", "лес", "россия", "зарубежье"],
    "шопинг": [],
    "праздники": ["новый год", "день победы", "праздник весны и труда (1 мая)", "пасха", "день знаний (1 сентября)", "международный женский день (8 марта)", "день защитника отечества (23 февраля)", "день святого валентина (14 февраля)", "рождество", "масленица", "хеллоуин"],
    "семья": [],
    "с друзьями": []
}

# Создание пользователей с интересами, созданными голсами и историями, лайками голсов и историй
def generate_users(num_users=100, num_posts=200, max_likes=20, max_stories_likes=50):
    users = []
    for user_id in range(num_users):
        interests = []
        for tag, subtags in possible_tags.items():
            if random.random() > 0.5:
                interests.append(tag)
                if subtags and random.random() > 0.5:
                    interests.extend(random.sample(subtags, random.randint(1, len(subtags))))
        created_posts = random.sample(range(num_posts), random.randint(0, 10))
        created_stories = []
        liked_posts = random.sample(range(num_posts), random.randint(0, min(num_posts // 5, max_likes)))
        # Случайное распределение с большей вероятностью нулевых лайков
        liked_stories_count = random.choices(range(max_stories_likes + 1), k=1, weights=[10] + [1]*max_stories_likes)[0]
        liked_stories = random.sample(range(num_stories), liked_stories_count)
        users.append({
            'id': user_id,
            'interests': interests,
            'created_posts': created_posts,
            'created_stories': created_stories,
            'liked_posts': liked_posts,
            'liked_stories': liked_stories
        })
    return pd.DataFrame(users)

# Создание постов с тегами и привязкой историй
def generate_posts_and_stories(num_posts=200, num_stories=500, num_users=100):
    posts = []
    stories = []
    for post_id in range(num_posts):
        tags = []
        for tag, subtags in possible_tags.items():
            if random.random() > 0.5:
                tags.append(tag)
                if subtags and random.random() > 0.5:
                    tags.extend(random.sample(subtags, random.randint(1, len(subtags))))
        tags = random.sample(tags, min(len(tags), 5))  # Ограничение на максимум 5 тегов
        post_stories = []
        author_id = random.randint(0, num_users - 1)
        published_at = datetime.now() - timedelta(days=random.randint(0, 30))  # Публикация в последние 30 дней
        num_post_stories = random.randint(1, 10)  # Увеличенный диапазон историй для каждого поста
        for _ in range(num_post_stories):
            story_id = len(stories)
            liked_by = random.sample(range(num_users), random.randint(0, num_users // 5))  # Ограничение количества лайков
            stories.append({
                'id': story_id,
                'goal_id': post_id,
                'likes': len(liked_by),
                'published_at': datetime.now() - timedelta(days=random.randint(0, 30)),
                'author_id': author_id,
                'liked_by': liked_by
            })
            post_stories.append(story_id)
            users.at[author_id, 'created_stories'].append(story_id)
            for user_id in liked_by:
                users.at[user_id, 'liked_stories'].append(story_id)
        posts.append({
            'id': post_id,
            'tags': tags,
            'stories': post_stories,
            'author_id': author_id,
            'published_at': published_at
        })
    return pd.DataFrame(posts), pd.DataFrame(stories)

# Генерация данных
num_stories = 500
users = generate_users(num_users=100, num_posts=200, max_likes=20, max_stories_likes=50)
posts, stories = generate_posts_and_stories(num_posts=200, num_stories=num_stories, num_users=100)

# Отображение данных
print(f"Пользователи:\n{users.head()}")
print(f"\nПосты:\n{posts.head()}")
print(f"\nИстории:\n{stories.head()}")


Пользователи:
   id                                          interests  \
0   0  [культура, туризм, лес, россия, пустыня, море,...   
1   1  [спорт, еда, кулинария, кофейня, бар, развлече...   
2   2  [спорт, хоббихорсинг, бильярд, бег, автоспорт,...   
3   3  [развлечения, кальян, танцы, образование, мита...   
4   4                [туризм, шопинг, семья, с друзьями]   

                                 created_posts  \
0  [149, 191, 72, 41, 182, 9, 52, 82, 131, 12]   
1      [90, 78, 57, 131, 160, 194, 15, 76, 11]   
2                                 [98, 26, 69]   
3     [37, 91, 135, 171, 61, 29, 132, 185, 65]   
4                 [2, 87, 72, 7, 195, 170, 64]   

                                     created_stories  \
0  [32, 33, 34, 35, 763, 764, 765, 766, 767, 984,...   
1  [412, 413, 414, 415, 416, 417, 418, 420, 421, ...   
2  [566, 567, 568, 569, 570, 571, 572, 573, 574, ...   
3                                                 []   
4                                           

In [11]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Подготовка данных для Surprise
def prepare_data_for_surprise(users, posts, stories):
    data = []
    for user_id in users['id']:
        for post_id in posts['id']:
            tags = posts.loc[posts['id'] == post_id, 'tags'].values[0]
            interests = users.loc[users['id'] == user_id, 'interests'].values[0]
            for tag in tags:
                data.append({
                    'user_id': user_id,
                    'post_id': post_id,
                    'tag': tag,
                    'interest': 1 if tag in interests else 0
                })
    df = pd.DataFrame(data)

    # Конвертация в формат Surprise
    reader = Reader(rating_scale=(0, 1))
    surprise_data = Dataset.load_from_df(df[['user_id', 'post_id', 'interest']], reader)

    trainset, testset = train_test_split(surprise_data, test_size=0.2)
    return trainset, testset

trainset, testset = prepare_data_for_surprise(users, posts, stories)


In [12]:
# Обучение модели SVD
def train_model_surprise(trainset):
    model = SVD()
    model.fit(trainset)
    return model

model_surprise = train_model_surprise(trainset)


In [13]:
# Оценка модели
def evaluate_model_surprise(model, testset):
    predictions = model.test(testset)
    accuracy_rmse = accuracy.rmse(predictions)
    accuracy_mae = accuracy.mae(predictions)
    print(f"RMSE: {accuracy_rmse}")
    print(f"MAE: {accuracy_mae}")

evaluate_model_surprise(model_surprise, testset)


RMSE: 0.4125
MAE:  0.3437
RMSE: 0.41251892723712447
MAE: 0.34374065995853414


In [14]:
# Генерация рекомендаций для пользователя
def generate_recommendations_surprise(user_id, model, posts, num_recommendations=5):
    recommendations = []
    for post_id in posts['id']:
        prediction = model.predict(user_id, post_id)
        if prediction.est >= 0.5:  # Пороговое значение для рекомендаций
            recommendations.append(post_id)
            if len(recommendations) >= num_recommendations:
                break
    return recommendations

# Пример рекомендаций для пользователя с id 0
recommendations_surprise = generate_recommendations_surprise(0, model_surprise, posts)
print(f"Рекомендации для пользователя 0: {recommendations_surprise}")


Рекомендации для пользователя 0: [8, 19, 33, 40, 102]


In [15]:
import joblib

# Сохранение модели
joblib.dump(model_surprise, 'model_surprise.pkl')


['model_surprise.pkl']